# Impulse responses to unexpected demographic shocks

This notebook is used to create simulations of unexpected shocks to the fertility rate or to life expectancy. It was created in response to a referee's request that we investigate uncertainty about demographic variables. 

## Fertility rate shock

The fertility shock scales down or up the fertility rates by age by a given proportion. The total shock is fed iterativelyl as a sequence of equal-sized small shocks over a fixed-length period. Expectations at each iteration put weights on staying at the current level or returning to the original level. The weight on staying follows an AR(1), so that a weight of 1 means that the shocks to date are seen as permanent. A weight arbitrarily close to 0 means that agents expect the fertility rates to jump back to their previous levels in the next period. 

## Life expectancy shock

The life expectancy shock is created by hastening the gains in life expectancy that were expected to occur over a longer period. For example, by assuming that life expectancy in 2020 matches what was expected to have occurred by 2070, we effectively add four years to life expectancy at birth. All unexpected gains in life expectancy are expected to be permanent, in contrast to our fertility shock for which we allowed the shocks to be temporary or permanent. The hastening of the gains is created by bringing forward the columns of the matrix of death rates. Once the shock is fully phased in, the standard gains per period are supposed to occur (with no gain beyond what would have been 2100).

## Plotting demographic suprises in United States and Japan

The first block of instructions below shows the size of the demographic surprises in Japan. These surprises were used as rough benchmarks for determining the size of the impulse responses of interest.

In [ ]:
using JLD, Plots, CSV, DataFrames, DelimitedFiles;
WPP_years = CSV.read("../Data/RawData/WPP_data_1992_2019.csv", DataFrame; header=false, skipto=2)[:,1]
WPP_data = Array{Float64,2}(readdlm("../Data/RawData/WPP_data_1992_2019.csv",',')[2:end,2:end])

fig_WPP_FR=plot(WPP_years,WPP_data[:,[1;4;2;5]],
#    xlabel = "Year",
    ylabel = "Chidren per woman",
    title = "Total fertility rate",
    label = ["U.S. (2019)" "U.S. (1992)" "Japan (2019)" "Japan (1992)"],
    color=[:blue :blue :red :red],    
    linestyle = [:solid :dash :solid :dash],
    linewidth = [4 4 4 4],
    xrotation=45,
    legend =:topright
)
#display(fig_WPP_FR)

fig_WPP_LE=plot(WPP_years,WPP_data[:,[7;10;8;11]],
#    xlabel = "Year",
    ylabel = "Years",
    title = "Life expectancy at birth",
    label = "",
    color=[:blue :blue :red :red],    
    linestyle = [:solid :dash :solid :dash],
    linewidth = [4 4 4 4],
    xrotation=45
)
#display(fig_WPP_LE)

fig_WPP_FR_LE = plot(fig_WPP_FR, fig_WPP_LE, layout=(2, 1),size = (600, 800), dpi=200)
display(fig_WPP_FR_LE)
savefig(fig_WPP_FR_LE, "../Figures/fig_WPP_FR_LE.png")

## Parameters of the IRF

In [ ]:
# AR(1) shocks to FR, no shock to LE (no kids)
simsuf = "_FR_AR1";
IRF_FR = 1;                # Indicator of fertility rate IRF
IRF_LE = 0;                # Indicator of life expectancy IRF
IRF_FR_Imp_siz = -0.333;   # Increase/Decrease ratio
IRF_FR_Imp_beg = 361;      # Period number, start of surprises
IRF_FR_Imp_end = 420;      # Period number, end of surprises
IRF_FR_Imp_rho = 0.95;     # AR parameter guiding expected return of fertility rates to original levels 
IRF_LE_beg = -999;         # Start period of surprises to life expectancy
IRF_LE_end = -999;         # End period of surprises to life expectancy
IRF_LE_tgt = -999;         # Period by 
filename = "../Rstar_sims/calibeta_epsilon0_eta76_dZ0_base.jld"

# Random walk shocks to FR, no shock to LE (no kids)
#simsuf = "_FR_RW";
#IRF_FR = 1;                # Indicator of fertility rate IRF
#IRF_LE = 0;                # Indicator of life expectancy IRF
#IRF_FR_Imp_siz = -0.333;   # Increase/Decrease ratio
#IRF_FR_Imp_beg = 361;      # Period number, start of surprises
#IRF_FR_Imp_end = 420;      # Period number, end of surprises
#IRF_FR_Imp_rho = 1;        # AR parameter guiding expected return of fertility rates to original levels 
#IRF_LE_beg = -999;         # Start period of surprises to life expectancy
#IRF_LE_end = -999;         # End period of surprises to life expectancy
#IRF_LE_tgt = -999;         # Period by 
#filename = "../Rstar_sims/calibeta_epsilon0_eta76_dZ0_base.jld"

# AR(1) shocks to FR, no shock to LE (with kids)
#simsuf = "_FR_AR1_kids";
#IRF_FR = 1;                # Indicator of fertility rate IRF
#IRF_LE = 0;                # Indicator of life expectancy IRF
#IRF_FR_Imp_siz = -0.333;   # Increase/Decrease ratio
#IRF_FR_Imp_beg = 361;      # Period number, start of surprises
#IRF_FR_Imp_end = 420;      # Period number, end of surprises
#IRF_FR_Imp_rho = 1;        # AR parameter guiding expected return of fertility rates to original levels 
#IRF_LE_beg = -999;         # Start period of surprises to life expectancy
#IRF_LE_end = -999;         # End period of surprises to life expectancy
#IRF_LE_tgt = -999;         # Period whose life expectancy corresponds to end of shock
#filename = "../Rstar_sims/calibeta_epsilon65_eta76_dZ0_dep.jld"

# Random walk shocks to FR, no shock to LE (with kids)
#simsuf = "_FR_RW_kids";
#IRF_FR = 1;                # Indicator of fertility rate IRF
#IRF_LE = 0;                # Indicator of life expectancy IRF
#IRF_FR_Imp_siz = -0.333;   # Increase/Decrease ratio
#IRF_FR_Imp_beg = 361;      # Period number, start of surprises
#IRF_FR_Imp_end = 420;      # Period number, end of surprises
#IRF_FR_Imp_rho = 0.95;     # AR parameter guiding expected return of fertility rates to original levels 
#IRF_LE_beg = -999;         # Start period of surprises to life expectancy
#IRF_LE_end = -999;         # End period of surprises to life expectancy
#IRF_LE_tgt = -999;         # Period by
#filename = "../Rstar_sims/calibeta_epsilon65_eta76_dZ0_dep.jld"

# No shock to FR, shock to LE (no kids)
#simsuf = "_LE";
#IRF_FR = 0;                # Indicator of fertility rate IRF
#IRF_LE = 1;                # Indicator of life expectancy IRF
#IRF_FR_Imp_siz = 999;      # Increase/Decrease ratio
#IRF_FR_Imp_beg = 999;      # Period number, start of surprises
#IRF_FR_Imp_end = 999;      # Period number, end of surprises
#IRF_FR_Imp_rho = 999;      # AR parameter guiding expected return of fertility rates to original levels 
#IRF_LE_beg = 341;          # Start period of surprises to life expectancy
#IRF_LE_end = 480;          # End period of surprises to life expectancy
#IRF_LE_tgt = 680;          # Period 
#filename = "../Rstar_sims/calibeta_epsilon0_eta76_dZ0_base.jld";


## Loading the packages, functions, and data

In [ ]:
# Installing packages if using them for the first time (can be done with other packages)
#using Pkg
#Pkg.add("JLD")   
#Pkg.add("Plots")
#Pkg.add("DataFrames")

# Loading packages, auxilliary function, type definitions
using JLD, Statistics, DelimitedFiles, Printf, LinearAlgebra, CSV, Plots, DataFrames;
include("typedefs.jl");
include("functionincludes.jl");

In [ ]:
# Loading data from baseline simulations
#filename = "../Rstar_sims/calibeta_epsilon0_eta76_dZ0_debug.jld"
solution_NOIRF = load(filename, "solution");
pars_NOIRF = load(filename, "pars");
simpars_NOIRF = load(filename, "simpars");
data_NOIRF = load(filename, "data");
Population_NOIRF = load(filename, "Population"); 
Dependents_NOIRF = load(filename, "Dependents");

# Creating structure that will contain simulation results
solution_IRF=deepcopy(solution_NOIRF)
simpars_IRF=deepcopy(simpars_NOIRF);
Population_IRF=deepcopy(Population_NOIRF)
Dependents_IRF=deepcopy(Dependents_NOIRF)
pars_IRF = deepcopy(pars_NOIRF)
data_IRF = deepcopy(data_NOIRF)

# Period indices
ageQ = 0.125:0.25:120;
periodQ = 1900.125:0.25:2400;

## Creating population under demographic surprise

The solution must be constructed for each of the period for which there is a demographic surprise. At each iteration:
1. We first replace the data in the demographic series that have changed, retaining the seeding period running from 1900:Q1 to 2099:Q4. 
2. We compute of a new set of demographic variables from the period of the surprise onward, extending the demographic data as needed beyond 2099:Q4 to use the standard solution programs. 
3. We reseed the starting conditions so that they match the solution prior to the surprise.
4. We compute the solution with the surprise then only takes demographic data 
5. We save the simulation results, overwriting the original results from the period of the surprise onward. 

We repeat the procedure iteratively until a new solution has been constructed for all periods with a demographic surprise. 

Before launching the iterative procedure, we recreate the population in the no-shock baseline to have the `Parent_child` array needed to track dependents. This matrix had not been saved in the baseline simulation. Before we proceed with the iterative procedure, we create copies of the data that will store the solution at each iteration for simulations. 

In [ ]:
# Creating the Parent_child structure for the no-shock population

# Declaring demographic simulation parameters (they should be verified against those used in simulation)
use_birth=false;
fix_birthgr=false;
migration=true;
extra_birthgr=0.0;

# Creating
(DependentsN, PopulationN, Parent_child_NOIRF, Parent_fertilityN) = Compute_population(pars_NOIRF, simpars_NOIRF, data_NOIRF, use_birth=use_birth, fix_birthgr=fix_birthgr, migration=migration, extra_birthgr=extra_birthgr);

# Comparing the population generated to the one imported
myper = 1:800
fig1=plot(periodQ[myper], [sum(Population_NOIRF,dims=1)[myper] sum(PopulationN,dims=1)[myper]]/1000000,
    xlabel = "Year",
    ylabel = "Persons (million)",
    title = "Population",
    label = ["Baseline" "Re-generated"],
    color=[:black :red],    
    linestyle = [:dash :dot],
    linewidth = [4 4],
    legend =:topleft
)
display(fig1)

# Clearing content of variables that are not needed
DependentsN = nothing;
Parent_fertilityN = nothing;
PopulationN = nothing;

Parent_child_IRF=deepcopy(Parent_child_NOIRF);

In [ ]:
# Creating structure that will contain simulation results at each iteration
solution_exp=deepcopy(solution_NOIRF);
simpars_exp=deepcopy(simpars_NOIRF);
data_exp=deepcopy(data_NOIRF)
pars_exp=deepcopy(pars_NOIRF)
pars_exp= reset_initial_conditions(pars_exp)  # Padding unecessary parameters with NaN as precaution against error

# Bounds of the shock periods
if (IRF_FR==1) & (IRF_LE==0)
    mint = IRF_FR_Imp_beg;
    maxt = IRF_FR_Imp_end;
elseif (IRF_FR==0) & (IRF_LE==1)
    mint = IRF_LE_beg;
    maxt = IRF_LE_end;
elseif (IRF_FR==1) & (IRF_LE==1)
    mint = min(IRF_FR_Imp_beg,IRF_LE_beg);
    maxt = max(IRF_FR_Imp_end,IRF_LE_end);
else
    error("Variable IRF_FR and/or IRF_LE not set properly.")
end
@printf("Start of simulation: %g\n",periodQ[mint]);
@printf("End of simulation  : %g\n",periodQ[maxt]);
if IRF_LE==1
    @printf("LE period target   : %g\n",periodQ[IRF_LE_tgt]);
    tmp1 = cumprod((1.0 .- data_NOIRF.death_rate).^(0.25),dims=1)
    tmp2 = [1.0 .- tmp1[1:1,:] ; tmp1[1:end-1,:] .- tmp1[2:end,:]]
    tmp_LE_NOIRF = (ageQ'*tmp2);
    @printf("LE at start          : %g years\n",tmp_LE_NOIRF[mint]);
    @printf("LE at end (original) : %g years\n",tmp_LE_NOIRF[maxt]);
    @printf("LE at end (target)   : %g years\n",tmp_LE_NOIRF[IRF_LE_tgt]);
    @printf("LE gain              : %g years\n",tmp_LE_NOIRF[IRF_LE_tgt]-tmp_LE_NOIRF[maxt]);
end

In [ ]:
#for t=mint:(mint+1)
for t=mint:maxt
    @printf("Iteration %g of %g\n", t-mint+1, maxt - mint +1)

    # STEP 1a: Adjusting the fertility rate assumptions
    if (IRF_FR) == 1 && (t>=IRF_FR_Imp_beg) && (t<=IRF_FR_Imp_end) 
        # Total shocks to datetmp0 = 1 + (t-IRF_FR_Imp_beg+1)/(IRF_FR_Imp_end-IRF_FR_Imp_beg+1)*IRF_FR_Imp_siz;
        tmp0 = 1 + (t-IRF_FR_Imp_beg+1)/(IRF_FR_Imp_end-IRF_FR_Imp_beg+1)*IRF_FR_Imp_siz;
        # AR(1) weight on no return
        tmp1 = ones(size(data_IRF.fertility_rate,1),1)*(IRF_FR_Imp_rho.^(0:(size(data_IRF.fertility_rate,2)-t))')
        data_IRF.fertility_rate = hcat(data_IRF.fertility_rate[:,1:(t-1)],tmp1.*(tmp0*data_NOIRF.fertility_rate[:,t:end]) + (1 .- tmp1).*data_NOIRF.fertility_rate[:,t:end]);
    end
    
    # STEP 1b: Adjusting the life expectancy assumptions
    if IRF_LE == 1 && (t>=IRF_LE_beg) && (t<=IRF_LE_end) 
        # Identifying the quarterly indexes in accelerated LE gains
        tmpi = Int64((IRF_LE_beg-1) + round((t-(IRF_LE_beg-1))/(IRF_LE_end-(IRF_LE_beg-1))*(IRF_LE_tgt-(IRF_LE_beg-1))))
        @printf("t=%g ; tmpi=%g\n per[t]=%g per[tmpi]=%g\n", t, tmpi, periodQ[t], periodQ[tmpi])
        data_IRF.death_rate[:,t:(size(data_IRF.death_rate,2)-(tmpi-t))] = data_NOIRF.death_rate[:,tmpi:end]
        if tmpi>t
            data_IRF.death_rate[:,end-(tmpi-t)+1:end] = data_NOIRF.death_rate[:,end]*ones(1,tmpi-t);
        end
    end
    
    # Visual checks of STEP 1
    # Check 1: impulse to average fertility
    myper = (mint-20):(maxt+140)
    figFR=plot(periodQ[myper], [sum(data_NOIRF.fertility_rate[:,myper],dims=1)' sum(data_IRF.fertility_rate[:,myper],dims=1)'],
    xlabel = "Period",
    ylabel = "Number of kids",
    title = "Raw fertility",
    label = ["Baseline" "Shock"],
    color=[:black :red],    
    linestyle = [:dash :dot],
    linewidth = [4 4],
    legend =:topleft
    )
    # Check 2: impulse to average life expectancy (assuming mortality rates in cross section)
    tmp1 = cumprod((1.0 .- data_NOIRF.death_rate).^(0.25),dims=1)
    tmp2 = [1.0 .- tmp1[1:1,:] ; tmp1[1:end-1,:] .- tmp1[2:end,:]]
    tmp_LE_NOIRF = (ageQ'*tmp2);
    tmp1 = cumprod((1.0 .- data_IRF.death_rate).^(0.25),dims=1)
    tmp2 = [1.0 .- tmp1[1:1,:] ; tmp1[1:end-1,:] .- tmp1[2:end,:]]
    tmp_LE_IRF = (ageQ'*tmp2)';
    figLE=plot(periodQ[myper], [tmp_LE_NOIRF[myper] tmp_LE_IRF[myper]],
    xlabel = "Period",
    ylabel = "Years",
    title = "Life expectancy",
    label = ["Baseline" "Shock"],
    color=[:black :red],    
    linestyle = [:dash :dot],
    linewidth = [4 4],
    legend =:topleft
    )
    fig_FR_LE_iter = plot(figFR, figLE, layout=(1, 2))
    display(fig_FR_LE_iter)


    # STEP 2: Expected demographic data from iteration period onward
    data_exp.death_rate = hcat(data_IRF.death_rate[:,t:end], data_IRF.death_rate[:,end]*ones(1,t-1)) 
    data_exp.fitted_age_marriage[:,1] = [data_IRF.fitted_age_marriage[t:end,1] ; data_IRF.fitted_age_marriage[end,1]*ones(t-1,1)]
    data_exp.births_interpolated .= NaN  # Not used in simulation
    data_exp.share_births_mothers = hcat(data_IRF.share_births_mothers[:,t:end], data_IRF.share_births_mothers[:,end]*ones(1,t-1))
    data_exp.net_migration_Q = hcat(data_IRF.net_migration_Q[:,t:end], data_IRF.net_migration_Q[:,end]*ones(1,t-1))
    data_exp.labendow = hcat(data_IRF.labendow[:,t:end], data_IRF.labendow[:,end]*ones(1,t-1))
    data_exp.fertility_rate = hcat(data_IRF.fertility_rate[:,t:end], data_IRF.fertility_rate[:,end]*ones(1,t-1));
    
    # STEP 3a: Re-seeding initial demographic variables
    data_exp.Population_end1899 .= Population_IRF[:,t-1]
    data_exp.Dependents_1899end[:,1] = Dependents_IRF[:,t-1]
    data_exp.Parent_child_1899end .= Parent_child_IRF[:,:,t-1]

    # STEP 3b: Re-seeding initial conditions (capital, MPK)
    # Note: Adults' capital holdings, K[:,t], are holdings start of t
    pars_exp.K=solution_IRF.K[solution_IRF.adultbeg:end,(t:t)];   # Passed through 4th argument of function
    pars_exp.R1900Q1=solution_IRF.R[t,1];   # Seed of R[1] in GE search, passed through 4th argument of function 
    pars_IRF.seedR2400[1] = solution_IRF.R[end,1]-0.005;    # Passed through 4th argument of function
    pars_IRF.seedR2400[2] = solution_IRF.R[end,1]+0.005;    # Passed through 4th argument of function

    # STEP 4: Computing solution with expected demographic data 
    (solution_exp,Population_exp,labendow_exp,death_rate_exp,Dependents_exp,Parent_child_exp) = GE_search_solution(pars_IRF,simpars_exp,data_exp,pars_exp,use_birth=use_birth,
                    fix_birthgr=fix_birthgr,migration=migration,extra_birthgr=extra_birthgr)
            
    # STEP 5a: Allocate equilibrium values to solution 
    solution_IRF.L[t:2000,1] = solution_exp.L[1:(2000-t+1)];
    solution_IRF.K[:,t:2000] = solution_exp.K[:,1:(2000-t+1)];
    solution_IRF.R[t:2000,1] = solution_exp.R[1:(2000-t+1),1];
    solution_IRF.W[t:2000,1] = solution_exp.W[1:(2000-t+1),1];
    solution_IRF.XI[t:2000,1] = solution_exp.XI[1:(2000-t+1), 1];
    solution_IRF.KLratio[t:2000,1] = solution_exp.KLratio[1:(2000-t+1),1];
    solution_IRF.KperAdult[t:2000,1] = solution_exp.KperAdult[1:(2000-t+1),1];
    solution_IRF.Chh_scaled[:, t:2000] = solution_exp.Chh_scaled[:,1:(2000-t+1)];
    solution_IRF.Chh_unscaled[:, t:2000] = solution_exp.Chh_unscaled[:, 1:(2000-t+1)];
    solution_IRF.ARR = 100.0*((1.0.+solution_IRF.R[:,1].-solution_IRF.delta).^4.0.-1.0)
    
    # STEP 5b: Allocate initial conditions values to solution 
    Population_IRF[:, t:2000] = Population_exp[:,1:(2000-t+1)];
    Dependents_IRF[:, t:2000] = Dependents_exp[:, 1:(2000-t+1)]
    Parent_child_IRF[:, :, t:2000] = Parent_child_exp[:,:,1:(2000-t+1)]
    
    
    # VISUAL CHECKS OF ITERATION
    # CHECK of individual capital holding with and without shock
    cc=1;  # Age of adult whose consuption we are looking at
    tt=mint;
    check_K_NOIRF = diag(solution_NOIRF.K[(pars_NOIRF.adultbeg+cc-1):pars_NOIRF.adultend,tt:(tt+pars_NOIRF.adultend-pars_NOIRF.adultbeg-cc+1)])
    check_K_IRF = diag(solution_IRF.K[(pars_IRF.adultbeg+cc-1):pars_IRF.adultend,tt:(tt+pars_IRF.adultend-pars_IRF.adultbeg-cc+1)])
    # Reporting capital holdings over life
    figK=plot((pars_NOIRF.adultbeg+cc-1):pars_NOIRF.adultend, [check_K_NOIRF check_K_IRF],
        xlabel = "Age (in years)",
        ylabel = "Capital holdings",
        title = "Asset holdings over time",
        label = ["Baseline" "Shock"],
        color=[:black :red],    
        linestyle = [:dash :dot],
#        ylims=(0.0, 2.5),
        linewidth = [4 4],
        legend =:topright
    )
    # Check equilibrium interest rate
    figARR=plot(periodQ[myper], [solution_NOIRF.ARR[myper] solution_IRF.ARR[myper]],
        xlabel = "Year",
        ylabel = "Percent",
        title = "Equilibrium real rate",
        label = ["Baseline" "Shock"],
        color=[:black :red],    
        linestyle = [:dash :dot],
        ylims=(0.0, 2.5),
        linewidth = [4 4],
        legend =:topright
    )
    fig_K_ARR = plot(figK, figARR, layout=(1, 2))
    display(fig_K_ARR)

end

In [ ]:
# Save simulation output
filename = "../Rstar_sims/IRF" * simsuf * ".jld"
jldopen(filename, "w") do file
    # IRF data
    write(file, "solution_IRF", solution_IRF)
    write(file, "pars_IRF", pars_IRF)
    write(file, "simpars_IRF", simpars_IRF)
    write(file, "data_IRF", data_IRF)
    write(file, "Population_IRF", Population_IRF); 
#    write(file, "Dependents_IRF", Dependents_IRF);
#    write(file, "Parent_child_IRF", Parent_child_IRF);
    # No IRF data
    write(file, "solution_NOIRF", solution_NOIRF);
    write(file, "pars_NOIRF", pars_NOIRF);
    write(file, "simpars_NOIRF", simpars_NOIRF);
    write(file, "data_NOIRF", data_NOIRF);
    write(file, "Population_NOIRF", Population_NOIRF); 
#    write(file, "Dependents_NOIRF", Dependents_NOIRF);
#    write(file, "Parent_child_NOIRF", Parent_child_NOIRF);
    # Control variables
    write(file, "IRF_FR_Imp_siz", IRF_FR_Imp_siz) 
    write(file, "IRF_FR_Imp_beg", IRF_FR_Imp_beg)
    write(file, "IRF_FR_Imp_end", IRF_FR_Imp_end)
    write(file, "IRF_FR_Imp_rho", IRF_FR_Imp_rho)
end

# Some checks

In [ ]:
# Exporting the last figure of demographic expectations
myper = (mint-40):(maxt+80)
figFR=plot(periodQ[myper], [sum(data_NOIRF.fertility_rate[:,myper],dims=1)' sum(data_IRF.fertility_rate[:,myper],dims=1)'],
    xlabel = "Year",
    ylabel = "Sum",
    title = "Raw fertility rate sum",
    label = ["Baseline" "Shock"],
    color=[:black :red],    
    linestyle = [:dash :dot],
    linewidth = [4 4],
    legend =:bottomleft
)
display(figFR)
savefig(figFR, "../Figures/Revision_IRF_FR_figFR" * simsuf * ".png")

In [ ]:
# Plotting the annualized real equilibrium rate and real GDP growth

# Computing the equilibrium rate and GDP growth
dY_NOIRF = exp.(solution_NOIRF.dZ) .* [1;solution_NOIRF.KLratio[2:end]./solution_NOIRF.KLratio[1:end-1]].^solution_NOIRF.alpha .* [1;solution_NOIRF.L[2:end]./solution_NOIRF.L[1:end-1]]
dY_NOIRF = 100*(dY_NOIRF.^4 .-1.0)
dY_IRF = exp.(solution_IRF.dZ) .* [1;solution_IRF.KLratio[2:end]./solution_IRF.KLratio[1:end-1]].^solution_IRF.alpha .* [1;solution_IRF.L[2:end]./solution_IRF.L[1:end-1]]
dY_IRF = 100*(dY_IRF.^4 .-1.0)

# Plotting
myper = (mint-40):(maxt+300)
figARR=plot(periodQ[myper], [solution_NOIRF.ARR[myper] solution_IRF.ARR[myper]],
    xlabel = "Year",
    ylabel = "Percent",
    title = "Equilibrium real rate",
    label = ["Baseline" "Shock"],
    color=[:black :red],    
    linestyle = [:dash :dot],
    ylims=(-1.0, 2.5),
    linewidth = [4 4],
    legend =:topright
)
display(figARR)

figdGDP = plot(periodQ[myper], [dY_NOIRF[myper] dY_IRF[myper] 0.0*dY_IRF[myper]],
    xlabel = "Year",
    ylabel = "Percent, annualized",
    title = "Real GDP growth",
    color=[:black :red :black],    
    linestyle = [:dash :dot :solid],
    ylims=(-1.0, 2.5),
    linewidth = [4 4 1],
#    label = ["Baseline" "Original TFP" "Corrected TFP"],
    label =""
)
display(figdGDP)

fig_ARR_dGDP = plot(figARR, figdGDP, layout=(1, 2),size = (800, 400), dpi=200)
#display(fig_ARR_dGDP)
savefig(fig_ARR_dGDP, "../Figures/Revision_IRF_FR_fig_ARR_dGDP" * simsuf * ".png")

In [ ]:
# Comparing the population generated to the one imported
myper = 300:600
figPOP=plot(periodQ[myper], [sum(Population_NOIRF,dims=1)[myper] sum(Population_IRF,dims=1)[myper]]/1000000,
    xlabel = "Year",
    ylabel = "Persons (million)",
    title = "Population",
    label = ["Baseline" "Shock"],
    color=[:black :red],    
    linestyle = [:dash :dot],
    linewidth = [4 4],
    legend =:topleft
)
display(figPOP)
figL=plot(periodQ[myper], [solution_NOIRF.L[myper] solution_IRF.L[myper]]/1000000,
    xlabel = "Year",
    ylabel = "Persons (million)",
    title = "Labor supply",
    label = ["Baseline" "Shock"],
    color=[:black :red],    
    linestyle = [:dash :dot],
    linewidth = [4 4],
    legend =:topleft
)
display(figL)

fig_POP_L = plot(figPOP, figL, layout=(1, 2),size = (800, 400), dpi=200)

savefig(fig_ARR_dGDP, "../Figures/Revision_IRF_FR_fig_POP_L" * simsuf * ".png")